### Planetscope preprocessing - Reflectance Images

This script:
- Reads multiple Planetscope images/scenes   

- Convert TOA Radiance to TOA Reflectance   

- Clips images over selected area of interest (AOI).   

Main packages used are rasterio and fiona.  

Code is based on notebooks from Planetlabs, available here: https://github.com/planetlabs/notebooks  

Data used are from Planetscope 21.-26. June 2019 over a selected study area in the Brazilian Amazon.   

Available export options:
- Reflectance scaled - *(default setting)*
- Reflectance scaled clipped to AOI - *(default setting)*   




- Reflectance unscaled
- Reflectance unscaled clipped to AOI
***

In [1]:
# Install libraries
import numpy as np
import matplotlib
import rasterio
import fiona

#### Step 1. Set up project folders - rest is automatic
The following input and directory structure is required:

- *Number of images/scenes*


- *Project folder*  -  all folders inside Project folder


- *AnalyticMS folder*  -  only the AnalyticMS.tif files, no unusable datamasks(udm)


- *Metadata folder*  -  only the metadata.xml files


- *AOI folder*  -  just one AOI file in GeoJSON format


- *Results folder*  -  receives export of reflectance images, also serves as input to the clip-functions that further clips the results to selected AOI


The default setting export scaled images and scaled images clipped to AOI.

The unscaled images and unscaled clipped images are available by calling the *unscaled-functions* in Step 6, then Step 8.

In [2]:
n_images = 6
Project_folder = 'planet_21-26/'
AnalyticMS_folder = 'planet_21-26/AnalyticMS_files/'
Metadata_folder = 'planet_21-26/Metadata_files/'
aoi_folder = 'planet_21-26/AOI/'
Results_folder = 'planet_21-26/Results/'

#### Step 2. Read and Inspect Images

In [3]:
# Use 6 scenes from 21.-26. June 2019 for coversion to TOA Reflectance
# Collect image filenames 
from os import walk
AnalyticMS_files = []
for (dirpath, dirnames, filenames) in walk(AnalyticMS_folder):
    AnalyticMS_files.extend(filenames)
    break

# Add image file path
AnalyticMS_path = []
for i in AnalyticMS_files:
    AnalyticMS_path.append(AnalyticMS_folder + i)
    print(AnalyticMS_folder + i) # Verify correct images

planet_21-26/AnalyticMS_files/20190721_133242_1034_3B_AnalyticMS.tif
planet_21-26/AnalyticMS_files/20190722_133028_103a_3B_AnalyticMS.tif
planet_21-26/AnalyticMS_files/20190723_131552_0f3d_3B_AnalyticMS.tif
planet_21-26/AnalyticMS_files/20190724_131505_1_1050_3B_AnalyticMS.tif
planet_21-26/AnalyticMS_files/20190725_133328_1042_3B_AnalyticMS.tif
planet_21-26/AnalyticMS_files/20190726_133225_103a_3B_AnalyticMS.tif


Note the ordering of the images: 1,2,3,4,5,6 for dates: 21,22,23,24,25,26

In [4]:
# Reading image metadata
Analytic_meta_files = []
for (dirpath, dirnames, filenames) in walk(Metadata_folder):
    Analytic_meta_files.extend(filenames)
    break

# Add image metadata file path
Analytic_meta_path = []
for i in Analytic_meta_files:
    Analytic_meta_path.append(Metadata_folder + i)
    print(Metadata_folder + i) # Verify correct metadata

planet_21-26/Metadata_files/20190721_133242_1034_3B_AnalyticMS_metadata.xml
planet_21-26/Metadata_files/20190722_133028_103a_3B_AnalyticMS_metadata.xml
planet_21-26/Metadata_files/20190723_131552_0f3d_3B_AnalyticMS_metadata.xml
planet_21-26/Metadata_files/20190724_131505_1_1050_3B_AnalyticMS_metadata.xml
planet_21-26/Metadata_files/20190725_133328_1042_3B_AnalyticMS_metadata.xml
planet_21-26/Metadata_files/20190726_133225_103a_3B_AnalyticMS_metadata.xml


In [5]:
# Loading images into list using rasterio
img_list = []
for image in AnalyticMS_path:
    img_list.append(rasterio.open(image))
print('Length of img_list: {} '.format(len(img_list)))

Length of img_list: 6 


In [6]:
# Inspecting image metadata
print("Image: dtype | crs | band count")
for image in img_list:
    print(image.meta['dtype'], image.meta['crs'], image.meta['count'])

Image: dtype | crs | band count
uint16 EPSG:32722 4
uint16 EPSG:32722 4
uint16 EPSG:32722 4
uint16 EPSG:32722 4
uint16 EPSG:32722 4
uint16 EPSG:32722 4


Let's take a closer look at what these bands contain:

In [7]:
# Read in color interpretations of each band in img1 - assume same values for rest of the images
colors = [img_list[0].colorinterp[band] for band in range(img_list[0].count)]

# taking a look at img1's band types:
for color in colors:
    print(color.name)

blue
green
red
undefined


The fourth channel is a binary alpha mask: this is common in satellite color models, and can be confirmed in Planet's documentation on the PSSCene3Band product.

#### Step 3. Extract the Data from Each Spectral Band
In this step, Rasterio (a Python library for reading and writing geospatial raster datasets) is used to open the raster images (the .tif files). 

Then, the band data will is extracted and loaded into arrays for further manipulation with Python's NumPy libary.

Note: in PlanetScope 4-band images, the band order is BGRN: (1) Blue, (2) Green, (3) Red, (4) Near-infrared.

In [8]:
# Radiance values are loaded into lists
band_blue_radiance = [0]*n_images
band_green_radiance = [0]*n_images
band_red_radiance = [0]*n_images
band_nir_radiance = [0]*n_images

# Using rasterio to read radiance values for the images
for i in range(n_images):
    with rasterio.open(AnalyticMS_path[i]) as src:
        band_blue_radiance[i] = src.read(1)
    with rasterio.open(AnalyticMS_path[i]) as src:
        band_green_radiance[i] = src.read(2)
    with rasterio.open(AnalyticMS_path[i]) as src:
        band_red_radiance[i] = src.read(3)
    with rasterio.open(AnalyticMS_path[i]) as src:
        band_nir_radiance[i] = src.read(4)

#### Step 4. Extract the Coefficients
Before converting to reflectance, the conversion coefficients from the metadata file (the .xml file) must be extracted.

In [9]:
from xml.dom import minidom

# Gathering the coefficients for the 7 images
coeffs_list = [] 
for image in Analytic_meta_path:
    xmldoc = minidom.parse(image)
    nodes = xmldoc.getElementsByTagName("ps:bandSpecificMetadata")
    
    # XML parser refers to bands by numbers 1-4
    coeffs = {} # Coefficients for each image/scene
    for node in nodes:
        band_nr = node.getElementsByTagName("ps:bandNumber")[0].firstChild.data
        if band_nr in ['1', '2', '3', '4']:
            i = int(band_nr)
            value = node.getElementsByTagName("ps:reflectanceCoefficient")[0].firstChild.data
            coeffs[i] = float(value)
    coeffs_list.append(coeffs)

for coffee in coeffs_list:
    print (coffee) # Verify data/coffee

{1: 2.22957954184e-05, 2: 2.36158292355e-05, 3: 2.63003228576e-05, 4: 3.95838798777e-05}
{1: 2.22520864862e-05, 2: 2.35908050459e-05, 3: 2.63407248495e-05, 4: 3.9577582354e-05}
{1: 2.3376622001e-05, 2: 2.47621088261e-05, 3: 2.76131113868e-05, 4: 4.14097363863e-05}
{1: 2.3524916513e-05, 2: 2.48470408363e-05, 3: 2.77309803512e-05, 4: 4.13036148082e-05}
{1: 2.21525543854e-05, 2: 2.34361488133e-05, 3: 2.61307246766e-05, 4: 3.90849967045e-05}
{1: 2.20519050955e-05, 2: 2.33785804456e-05, 3: 2.61037617704e-05, 4: 3.92215395407e-05}


Note that the coefficients are all of order 1e<sup>-5</sup>, and that the coefficient for NIR is significantly higher than the coefficient for blue. This is a big deal if your use case involves performing band math because a pixel with a NIR/blue ratio of 1.0 in the radiance image will have a NIR/blue ratio of 3.35/1.929=1.73 in the reflectance image.   
Most spectral indices are defined in terms of reflectance, not radiance.

#### Step 5: Convert Radiance to Reflectance
Radiance is measured in SI units: $W/m^2$. Reflectance is a ratio from 0 to 1. The conversion is performed as a per-band scalar multiplication:

In [10]:
# Radiance values are loaded into lists
band_blue_reflectance = [];   blue_ref_scaled = [0]*n_images
band_green_reflectance = [];  green_ref_scaled = [0]*n_images
band_red_reflectance = [];    red_ref_scaled = [0]*n_images
band_nir_reflectance = [];    nir_ref_scaled = [0]*n_images

# Calculating reflectance from radiance and conversion coefficients
for i in range(n_images):
        band_blue_reflectance.append(band_blue_radiance[i] * coeffs_list[i][1])
        band_green_reflectance.append(band_green_radiance[i] * coeffs_list[i][2])
        band_red_reflectance.append(band_red_radiance[i] * coeffs_list[i][3])
        band_nir_reflectance.append(band_nir_radiance[i] * coeffs_list[i][4])
        
# Check the results by looking at the min-max range of the radiance (before) vs reflectance (after)
# for the red band from the first image, excluding NaN values
red_rad_min = np.nanmin(band_red_radiance[0]);    red_rad_max = np.nanmax(band_red_radiance[0])
red_ref_min = np.nanmin(band_red_reflectance[0]); red_ref_max = np.nanmax(band_red_reflectance[0])
print("Red band radiance goes from: {} to {}".format(red_rad_min, red_rad_max))
print("Red band reflectance goes from: {} to {}".format(red_ref_min, red_ref_max))

Red band radiance goes from: 0 to 12546
Red band reflectance goes from: 0.0 to 0.3299638505714496


#### Step 6. Write the Reflectance Image
A note: Reflectance is generally defined as a floating point number between 0 and 1, but image file formats are much more commonly stored as unsigned integers. A common practice in the industry is to multiply the reflectance value by a *scale factor* of 10,000, then save the result as a file with data type uint16.


Export of normal scaled GeoTIFF (multiplied with scale factor of 10 000 and stored as *dtype=uint16*)

In [11]:
# Function to export scaled images - (alternative option)
def scaled_export(images):
    for i in range(len(images)):
        # Get the metadata of original GeoTIFF:
        meta = images[i].meta
        
        # Set the source metadata as kwargs we'll use to write the new data:
        # update the 'dtype' value to uint16:
        kwargs = meta
        kwargs.update(dtype='uint16')
        
        # As noted above, scale reflectance value by a factor of 10k:
        scale = 10000
        blue_ref_scaled[i] = scale * band_blue_reflectance[i]
        green_ref_scaled[i] = scale * band_green_reflectance[i]
        red_ref_scaled[i] = scale * band_red_reflectance[i]
        nir_ref_scaled[i] = scale * band_nir_reflectance[i]
        
        # Compute new min & max values for the scaled red band, just for comparison
        red_min_scaled = np.amin(red_ref_scaled[0])
        red_max_scaled = np.amax(red_ref_scaled[0])
        
        # Convert to type 'uint16'
        from rasterio import uint16
        blue_scaled = blue_ref_scaled[i].astype(uint16)
        green_scaled = green_ref_scaled[i].astype(uint16)
        red_scaled = red_ref_scaled[i].astype(uint16)
        nir_scaled = nir_ref_scaled[i].astype(uint16)
        
        # New name for exported image
        img_name_ref_scaled = (Results_folder + AnalyticMS_files[i]).replace('.tif','_Ref.tif')
        
        # Write band calculations to a new unscaled GeoTIFF file with same band order (BGRN)
        with rasterio.open(img_name_ref_scaled, 'w', **kwargs) as dst:
                dst.write_band(1, blue_scaled)
                dst.write_band(2, green_scaled)
                dst.write_band(3, red_scaled)
                dst.write_band(4, nir_scaled)
         # Comparing min & max values before/after scaling, with the first image red band
        if i == 0:
            print("With scale factor: {}".format(scale))
            print("Before scaling: Red band reflectance from: {} to {}"\
                  .format(red_ref_min, red_ref_max))
            print("After scaling: Red band reflectance from: {} to {}"\
                  .format(red_min_scaled,red_max_scaled))
        if i == (len(images)-1):
            print("Success, {} scaled images(dtype={}) have been exported to your Results folder: {}"\
                  .format(n_images, kwargs['dtype'], Results_folder ))

Export of unscaled GeoTIFF, dtype=float64 with Reflectance values between 0.0 - 1.0

In [12]:
# Function to export unscaled images - (alternative option)
def unscaled_export(images):
    for i in range(len(images)):
        # Get the metadata of original GeoTIFF:
        meta = images[i].meta

        # Set the source metadata as kwargs we'll use to write the new data:
        # update the 'dtype' value to float64:
        kwargs = meta
        kwargs.update(dtype='float64')

        # New name for exported image
        img_name_ref_unscaled = (Results_folder + AnalyticMS_files[i]).replace('.tif','_Ref_unscaled.tif')

        # Write band calculations to a new unscaled GeoTIFF file with same band order (BGRN)
        with rasterio.open(img_name_ref_unscaled, 'w', **kwargs) as dst:
                dst.write_band(1, band_blue_reflectance[i])
                dst.write_band(2, band_green_reflectance[i])
                dst.write_band(3, band_red_reflectance[i])
                dst.write_band(4, band_nir_reflectance[i])
        
        if i == (len(images)-1):
            print("Success, {} unscaled images(dtype={}) have been exported to your Results folder: {}"\
                  .format(n_images, kwargs['dtype'], Results_folder ))

##### Export Reflectance Images to your Results folder
- For the scaled images, run the function *scaled_export()* with input(*img_list*)   


- For the unscaled images, run the function *unscaled_export()* with input(*img_list*)

Note that unscaled(*dtype=float64*) takes 4 times as much storage space as scaled(*dtype=uint16*)

For clipping reflectance to AOI, proceed further

In [13]:
# Uncomment and run for wanted image type:
# Export scaled images:
scaled_export(img_list)

# Export unscaled images:
#unscaled_export(img_list)

With scale factor: 10000
Before scaling: Red band reflectance from: 0.0 to 0.3299638505714496
After scaling: Red band reflectance from: 0.0 to 3299.638505714496
Success, 6 scaled images(dtype=uint16) have been exported to your Results folder: planet_21-26/Results/


In [14]:
### Load image results into lists for the clip functions
Results_files = []
Results_path_scaled = []; Results_path_unscaled = []
img_list_scaled = []; img_list_unscaled = []

for (dirpath, dirnames, filenames) in walk(Results_folder):
    Results_files.extend(filenames)
    break

for filename in Results_files:
    if "Ref.tif" in filename:
        Results_path_scaled.append(Results_folder + filename)
        img_list_scaled.append(rasterio.open(Results_folder + filename))
    if "Ref_unscaled.tif" in filename:
        Results_path_unscaled.append(Results_folder + filename)
        img_list_unscaled.append(rasterio.open(Results_folder + filename))

Using rasterio.plot (a matplotlib interface) to preview the results of our mosaic.

In [15]:
from rasterio.plot import show
#show(img_list[0])

#### Step 7. Clip Images to AOI Boundaries
A mask is used to clip the images to the extents of our AOI.

For this operation rasterio's sister-library fiona will read in our AOI (as a GeoJSON file).
Just as rasterio is used to manipulate raster data, fiona works similarly on vector data. Where rasterio represents raster imagery as numpy arrays, fiona represents vector data as GeoJSON-like Python dicts.

After the GeoJSON is read, the geometry of the AOI is extracted, with the geometry as dict key.

##### A note about Coordinate Reference Systems

Before the AOI can be applied to the images, the Coordinate Reference System (CRS) need to match.
This can be checked by reading the crs attribute of the Collection object generated by *fiona.open()*.

In this example, the CRS of the AOI is: *EPSG:4326*.

While the CRS for the images is: *EPSG:32722*.


Before the clip can be applied, the geometry of the AOI needs to be transformed to match the CRS of the imagery.
Luckily, fiona is smart enough to apply the necessary mathematical transformation to a set of coordinates in order to convert them to new values: 
apply *fiona.transform.transform_geom* to the AOI geometry to do this, specifying the GeoJSON's CRS as the source CRS, and the imagery's CRS as the destination CRS.

In [16]:
# Reading AOI filename - (assumes just one file in the AOI folder)
for (dirpath, dirname, filename) in walk(aoi_folder):
    aoi_filename = filename[0]
    break
    
# Add file path
aoi_path = aoi_folder + aoi_filename
print(aoi_path) # Verify correct AOI file

planet_21-26/AOI/AOI_small.geojson


In [17]:
# Loading CRS of original images and AOI into corresponding lists
img_list_crs = []
aoi_list_crs = [0]*n_images
aoi_info = fiona.open(aoi_path)
    
# Checking CRS of images and AOI
print("Image CRS:")
for i in range(len(img_list)):
    img_list_crs.append(img_list[i].meta['crs'])
    aoi_list_crs[i] = aoi_info.crs['init']
    print(img_list[i].meta['dtype'], img_list[i].meta['crs'], img_list[i].meta['count'])

print('-------------------\n' "AOI CRS: {}".format(aoi_list_crs[0]))
print(aoi_list_crs)

Image CRS:
uint16 EPSG:32722 4
uint16 EPSG:32722 4
uint16 EPSG:32722 4
uint16 EPSG:32722 4
uint16 EPSG:32722 4
uint16 EPSG:32722 4
-------------------
AOI CRS: epsg:4326
['epsg:4326', 'epsg:4326', 'epsg:4326', 'epsg:4326', 'epsg:4326', 'epsg:4326']


In [18]:
# Create matching AOI CRS for each image CRS
aoi_crs_transformed = []

for i in range(len(img_list)):
    # Use fiona to open the original AOI GeoJSON
    with fiona.open(aoi_path) as mt:
        aoi = [feature["geometry"] for feature in mt]

    # Transfrom AOI CRS to match image CRS
    from fiona.transform import transform_geom
    transformed_coords = transform_geom(str(aoi_list_crs[i]), str(img_list_crs[i]), aoi[0])
    aoi = [transformed_coords]
    aoi_crs_transformed.append([transformed_coords])

At this stage the AOI geometry is read and the coordinates have been transformed to match the images.
Next, *rasterio.mask.mask* will create a mask over our images, using the AOI geometry as the mask line.

Passing *crop=True* to the mask function will automatically crop the bits of our images that fall outside the mask boundary.

#### Step 8. Export results
Lastly, the result is saved as a final output GeoTIFF, scaled and clipped to selected AOI.

In [19]:
# Import rasterio's mask tool
from rasterio.mask import mask

# Loading clipped images into list
img_list_clipped = [0]*n_images

# Export reflectance images, scaled and clipped to AOI
def scale_clip_export(images):
    for i in range(len(images)):
        
        with rasterio.open(Results_path_scaled[i]) as image:
            clipped, transform = mask(image, aoi, crop=True)
            img_list_clipped[i] = clipped
        
        # Use metadata from our original images
        meta = images[i].meta.copy()

        # Update metadata with new, clipped image boundaries
        meta.update({"transform": transform,
            "height":clipped.shape[1],
            "width":clipped.shape[2]})
        
        # Set the source metadata as kwargs we'll use to write the new data:
        # update the 'dtype' value from uint16 to float64:
        kwargs = meta
        kwargs.update(dtype='uint16')
        
        # New image name for export
        img_name_ref_scale_clip = (Results_folder + AnalyticMS_files[i]).replace('.tif','_Ref_clip.tif')

        # Writes the new images into project folder
        with rasterio.open(img_name_ref_scale_clip,'w', **kwargs) as dst:
            dst.write(clipped)
            
        if i == (len(images)-1):
                print("Success, {} scaled clipped images(dtype={}) have been exported to "\
                "your Results folder: {}".format(n_images, kwargs['dtype'], Results_folder))

In [20]:
# Loading clipped images into list
img_list_clipped = [0]*n_images

# Export reflectance images, unscaled and clipped to AOI
def unscale_clip_export(images):
    for i in range(len(images)):
        
        with rasterio.open(Results_path_unscaled[i]) as image:
            clipped, transform = mask(image, aoi, crop=True)
            img_list_clipped[i] = clipped
        
        # Use metadata from our original images
        meta = images[i].meta.copy()

        # Update metadata with new, clipped image boundaries
        meta.update({"transform": transform,
            "height":clipped.shape[1],
            "width":clipped.shape[2]})
        
        # Set the source metadata as kwargs we'll use to write the new data:
        # update the 'dtype' value from uint16 to float64:
        kwargs = meta
        kwargs.update(dtype='float64')
        
        # New image name for export
        img_name_ref_scale_clip = (Results_folder + AnalyticMS_files[i]).\
        replace('.tif','_Ref_clip_unscaled.tif')

        # Writes the new images into project folder
        with rasterio.open(img_name_ref_scale_clip,'w', **kwargs) as dst:
            dst.write(clipped)
            
        if i == (len(images)-1):
                print("Success, {} unscaled clipped images(dtype={}) have been exported to "\
                "your Results folder: {}".format(n_images, kwargs['dtype'], Results_folder))

In [21]:
# Watch the results
#show(img_list_clipped[0])

##### Export clipped Reflectance Images to your project folder
- For the scaled clipped images, run the function *scale_clip_export()* with input(*img_list*)   


- For the unscaled clipped images, run the function *unscale_clip_export()* with input(*img_list*)

Note that unscaled(*dtype=float64*) takes 4 times as much storage space as scaled(*dtype=uint16*)

In [24]:
# Uncomment and run for wanted image type:
# Export scaled images: - (default setting)
scale_clip_export(img_list)

# Export unscaled images:
#unscale_clip_export(img_list)

Success, 6 scaled clipped images(dtype=uint16) have been exported to your Results folder: planet_21-26/Results/
